If you're like me and enjoy the false and fleeting impression of superiority that comes when you know a word someone else doesn't. If you're like me and go to parties for the sole purpose of seeking victims to get a one-up on, here's a cool word to add to your arsenal:

**demonym:** a noun used to denote the natives or inhabitants of a particular country, state, city, etc.
_"he struggled for the correct demonym for the people of Manchester"_

During a discussion (about traveling in Europe) someone said "why are the swiss so miserable". Now, I wouldn't say that the swiss were especially miserable (a couple of ex-girlfriends aside), but to be fair he was contrasting with Italians, so perhaps he has a point. I apologize if you are swiss, or one of the two ex-girlfriends -- nothing personal, this is all for effect. 

We googled "why are the swiss so ", and sure enough, "why are the swiss so miserable" came up as one of the suggestions. So we got curious and started googling other peoples: the French, the Germans, etc.

That's the back-story of this analysis. This analysis is meant to get an idea of what we think of peoples from other countries. Of course, one can rightfully critique the approach I'll take to gauge "what we think" -- all three of these words should, but will not, be defined. I'm just going to see what google's *current* auto-suggest comes back with when I enter "why are the X so " (where X will be a noun that denotes the natives of inhabitants of a particular country; a *demonym* if you will). 

Again, word of warning: All data and analyses are biased. Take everything you'll read here (and to be fair, what you read anywhere) with a grain of salt. For simplicitly I'll say "what we think of...", but I don't **really** mean that.

Resources:

* http://www.geography-site.co.uk/pages/countries/demonyms.html for my list of demonyms.
* google for my suggestion engine.

In [503]:
the_results_are = """
afghan                         eyes beautiful
albanian                            beautiful
american                 girl dolls expensive
australian                               tall
belgian                            fries good
bhutanese                               happy
brazilian                    good at football
british             full of grief and despair
bulgarian                    properties cheap
burmese                     cats affectionate
cambodian                         cows skinny
canadian                                 nice
chinese                               healthy
colombian                        avocados big
cuban                             cigars good
czech                                    tall
dominican        republic and haiti different
egyptian                       gods important
english                              reserved
eritrean                            beautiful
ethiopian                           beautiful
filipino                                proud
finn                          shoes expensive
french                                healthy
german                                   tall
greek                          gods messed up
haitian                        parents strict
hungarian                          words long
indian                     tv debates chaotic
indonesian                              smart
iranian                             beautiful
israeli                   startups successful
italian                                 short
jamaican                       sprinters fast
japanese                               polite
kenyan                           runners good
lebanese                                 rich
malagasy                           names long
malaysian                         drivers bad
maltese                                  rude
mongolian                        horses small
moroccan                       rugs expensive
nepalese                            beautiful
nigerian                                 tall
north korean                         hats big
norwegian                       flights cheap
pakistani                                fair
peruvian                      blueberries big
pole                             vaulters hot
portuguese                              short
puerto rican          and cuban flags similar
romanian                            beautiful
russian                          good at math
samoan                                    big
saudi                                arrogant
scottish                               bitter
senegalese                               tall
singaporean                              rude
somali                         parents strict
south african                       plugs big
""";

A few notes:
* It's not "what we think", but what google auto-suggest yields, now, from my IP, with my browser, etc.
* Here I'm showing only the first item of the results I got -- some demonyms (look it up) yield 20 results (for swiss, singaporean, british, french, and german namely -- the most "talked about" peoples), and others only 1.
* Well, in fact, some yield 0 results, and those are not shown in the list of 72 demonyms here.
* Depending on how exactly you structure your query, you'll get different results. Here I used "why are the X so " ("not why are Xs so", or "why are the X", etc.)
* You'll notice that not all the results are characteristics of people of a nation (example pole: vaulters hot")

In [546]:
import os
import re
from collections import Counter
import pandas as pd
import random


# Analysis

## Data source (after acquisition)

In [ ]:
# from tapyoca.demonyms.data_acquisition import suggestions_df
# sdff = suggestions_df()

In [562]:
# sdff = pd.read_pickle('demonym_suggested_characteristics.p')

In [518]:
import os
import re
from py2store import *
from collections import Counter
import pandas as pd

root = os.path.expanduser('~/ddir/demonym_adjectives/so')
s = QuickJsonStore(root)
print(len(s))

sdf = list()
for demonym, r in s.items():
    suggestions = r[1]
    if len(suggestions) > 0:
        relevances = r[-1].get('google:suggestrelevance', [])
        assert len(suggestions) == len(relevances)
        sdf.extend([{'demonym': demonym, 'suggestion': t, 'relevance': tt} for t, tt in zip(suggestions, relevances)])
sdf = pd.DataFrame(sdf)
sdf['demonym'] = list(map(str.lower, sdf['demonym']))
print(sdf.shape)
sdf


prefix_templates_1 = 'why are the {demonym}'
prefix_templates_2 = 'why are the {demonym}|why are {demonym}'
prefix_templates_3 = 'why are the {demonym}|why are {demonym}|why {demonym}'

def remove_query_prefix(demonym, query, prefix_template=prefix_templates_3):
    demonym = demonym.lower()
    t = re.sub(prefix_template.format(demonym=demonym), '', query)
    return t

sdf['characteristic'] = list(map(lambda x: remove_query_prefix(*x), zip(sdf.demonym, sdf.suggestion)))
lidx = list(map(lambda x: x[0] != x[1], zip(sdf.characteristic, sdf.suggestion)))
sdff = sdf.iloc[lidx].reset_index(drop=True)
print(sdff.shape)

so_p = re.compile('so\ ')

def post_process_characteristics(t):
    t = t.strip()
    if t.startswith('s '):
        t = t[len('s '):]
    t = so_p.sub('', t)
    return t
    
sdff['characteristic'] = [post_process_characteristics(t) for t in sdff['characteristic']]
sdff = sdff.sort_values(by=['demonym', 'relevance'], ascending=[True, False]).reset_index(drop=True)

# sdff.to_pickle('demonym_suggested_characteristics.p')
sdff

215
(678, 3)
(523, 4)


,demonym,suggestion,relevance,characteristic
0,afghan,why are afghan eyes so beautiful,601,eyes beautiful
1,afghan,why are afghan biscuits so called,600,biscuits called
2,afghan,why are afghan blankets so warm,552,blankets warm
3,afghan,why are afghan rugs so expensive,551,rugs expensive
4,afghan,why afghanistan so poor,550,istan poor
...,...,...,...,...
518,welsh,why are welsh words so long,553,words long
519,welsh,why are welsh names so long,552,names long
520,welsh,why are welsh hats so tall,551,hats tall
521,welsh,why are welsh corgis so expensive,550,corgis expensive


In [519]:
sdff[sdff['demonym'] == 'israeli']  # Note that the "so" of the query was misunterpreted as the suffix of a word

,demonym,suggestion,relevance,characteristic
253,israeli,why are israeli startups so successful,600,startups successful
254,israeli,why are israeli helmets so big,551,helmets big


In [520]:
sdff[sdff['demonym'] == 'swiss']

,demonym,suggestion,relevance,characteristic
446,swiss,why are the swiss so good at making watches,601,good at making watches
447,swiss,why are the swiss so healthy,600,healthy
448,swiss,why are the swiss so miserable,567,miserable
449,swiss,why are the swiss so happy,566,happy
450,swiss,why are the swiss so boring,565,boring
451,swiss,why are the swiss so good at making chocolate,564,good at making chocolate
452,swiss,why are the swiss so thin,563,thin
453,swiss,why are the swiss so neutral,562,neutral
454,swiss,why are the swiss so precise,561,precise
455,swiss,why are the swiss so efficient,560,efficient


## Analyses

In [555]:
print(sdff.demonym.unique().tolist())

['afghan', 'albanian', 'american', 'australian', 'belgian', 'bhutanese', 'brazilian', 'british', 'bulgarian', 'burmese', 'cambodian', 'canadian', 'chinese', 'colombian', 'cuban', 'czech', 'dominican', 'egyptian', 'english', 'eritrean', 'ethiopian', 'filipino', 'finn', 'french', 'german', 'greek', 'haitian', 'hungarian', 'indian', 'indonesian', 'iranian', 'israeli', 'italian', 'jamaican', 'japanese', 'kenyan', 'lebanese', 'malagasy', 'malaysian', 'maltese', 'mongolian', 'moroccan', 'nepalese', 'nigerian', 'north korean', 'norwegian', 'pakistani', 'peruvian', 'pole', 'portuguese', 'puerto rican', 'romanian', 'russian', 'samoan', 'saudi', 'scottish', 'senegalese', 'serbian', 'singaporean', 'somali', 'south african', 'south korean', 'sri lankan', 'sudanese', 'swiss', 'syrian', 'taiwanese', 'thai', 'tongan', 'ukrainian', 'vietnamese', 'welsh', 'zambian']


### Prep to save stuff

In [521]:
xls_file = 'what_we_think_about_demonyns.xlsx'
xls_writer = pd.ExcelWriter(xls_file, engine='xlsxwriter')

def save_to_xls(df, sheet_name):
    df.to_excel(xls_writer, sheet_name=sheet_name, index=False)
#     xls_writer.save()
    
def write_sr_to_sheet(sr, index_name='demonym', val_name='count', sheet_name=None):
    sr.index.name = index_name
    sr.name = val_name
    sheet_name = sheet_name or f"{index_name}_{val_name}"
    sr.reset_index().to_excel(xls_writer, sheet_name=sheet_name, index=False)

### Save the source data

In [522]:
save_to_xls(sdff, sheet_name='all')

### Demonyms covered

In [523]:
demonym_count = pd.Series(Counter(sdff.demonym)).sort_values(ascending=False)
write_sr_to_sheet(demonym_count, index_name='demonym', val_name='count')
print(f"{len(demonym_count)} 'demonyms' covered")

73 'demonyms' covered


In [526]:
print("Those that people we most talk about:")
demonym_count.head((16))

Those that people we most talk about:


demonym
french         20
singaporean    20
german         20
british        20
swiss          20
english        19
italian        18
cuban          18
canadian       18
welsh          18
australian     17
maltese        16
american       16
japanese       14
scottish       14
belgian        13
Name: count, dtype: int64

In [528]:
t = len(demonym_count[demonym_count == 1])
print(f"Those {t} demonyms that people don't inquire much")
demonym_count[demonym_count == 1]

Those 19 demonyms that people don't inquire much


demonym
somali          1
bhutanese       1
syrian          1
tongan          1
cambodian       1
malagasy        1
saudi           1
serbian         1
czech           1
eritrean        1
finn            1
puerto rican    1
pole            1
haitian         1
hungarian       1
peruvian        1
moroccan        1
mongolian       1
zambian         1
Name: count, dtype: int64

### Characteristics

How do people perceive others in the world? 

More precisely, what are the words they use to describe them?

In [529]:
feature_count = pd.Series(Counter(sdff['characteristic'])).sort_values(ascending=False)
write_sr_to_sheet(feature_count, index_name='characteristic', val_name='count')
print(f"The data shows that people use a set of {len(feature_count)}", 
      "words or expressions when asking google why such and such is so...")

The data shows that people use a set of 366 words or expressions when asking google why such and such is so...


In [531]:
n = 13
print(f"The top {n} words that are used are")
feature_count.head(n)

The top 13 words that are used are


characteristic
beautiful         11
tall              11
short              9
names long         8
proud              8
parents strict     8
smart              8
nice               7
boring             6
rich               5
dark               5
successful         5
skinny             4
Name: count, dtype: int64

### Who is...

In [535]:
def whois(characteristic):
    return list(sdff[sdff['characteristic'] == characteristic]['demonym'])

In [536]:
whois('beautiful')

['albanian',
 'eritrean',
 'ethiopian',
 'filipino',
 'iranian',
 'lebanese',
 'nepalese',
 'pakistani',
 'romanian',
 'ukrainian',
 'vietnamese']

In [556]:
whois_sr = dict()
for characteristic in feature_count.index.values:
    whois_sr[characteristic] = ', '.join(whois(characteristic))
whois_sr = pd.Series(whois_sr)
write_sr_to_sheet(whois_sr, index_name='characteristic', val_name='demonyms')

for characteristic, demonyms in whois_sr.head(12).items():
    print(f"{characteristic:<15}: {demonyms}")

beautiful      : albanian, eritrean, ethiopian, filipino, iranian, lebanese, nepalese, pakistani, romanian, ukrainian, vietnamese
tall           : australian, czech, german, nigerian, pakistani, samoan, senegalese, serbian, south korean, sudanese, taiwanese
short          : filipino, indonesian, italian, maltese, nepalese, pakistani, portuguese, singaporean, welsh
names long     : indian, malagasy, nigerian, portuguese, russian, sri lankan, thai, welsh
proud          : albanian, ethiopian, filipino, iranian, lebanese, portuguese, scottish, welsh
parents strict : albanian, ethiopian, haitian, indian, lebanese, pakistani, somali, sri lankan
smart          : indonesian, iranian, lebanese, pakistani, romanian, singaporean, taiwanese, vietnamese
nice           : canadian, english, filipino, nepalese, portuguese, taiwanese, thai
boring         : british, english, french, german, singaporean, swiss
rich           : lebanese, pakistani, singaporean, taiwanese, vietnamese
dark           : filip

### Top hit

In [557]:
t = sdff[['demonym', 'characteristic']].groupby('demonym').apply(lambda x: x.iloc[0])['characteristic']
write_sr_to_sheet(t, index_name='demonym', val_name='top characteristic')
print(t.head(10))

demonym
afghan                   eyes beautiful
albanian                      beautiful
american           girl dolls expensive
australian                         tall
belgian                      fries good
bhutanese                         happy
brazilian              good at football
british       full of grief and despair
bulgarian              properties cheap
burmese               cats affectionate
Name: top characteristic, dtype: object


In [558]:
for demonym, charact in t.iteritems():
    print(f"{demonym:<10}\t{charact:>30}")

afghan    	                eyes beautiful
albanian  	                     beautiful
american  	          girl dolls expensive
australian	                          tall
belgian   	                    fries good
bhutanese 	                         happy
brazilian 	              good at football
british   	     full of grief and despair
bulgarian 	              properties cheap
burmese   	             cats affectionate
cambodian 	                   cows skinny
canadian  	                          nice
chinese   	                       healthy
colombian 	                  avocados big
cuban     	                   cigars good
czech     	                          tall
dominican 	  republic and haiti different
egyptian  	                gods important
english   	                      reserved
eritrean  	                     beautiful
ethiopian 	                     beautiful
filipino  	                         proud
finn      	               shoes expensive
french    	                       

In [559]:
t.shape

(73,)

## Persist the xls

In [560]:
xls_writer.save()

,demonym,suggestion,relevance,characteristic
0,afghan,why are afghan eyes so beautiful,601,eyes beautiful
1,afghan,why are afghan biscuits so called,600,biscuits called
2,afghan,why are afghan blankets so warm,552,blankets warm
3,afghan,why are afghan rugs so expensive,551,rugs expensive
4,afghan,why afghanistan so poor,550,istan poor
...,...,...,...,...
518,welsh,why are welsh words so long,553,words long
519,welsh,why are welsh names so long,552,names long
520,welsh,why are welsh hats so tall,551,hats tall
521,welsh,why are welsh corgis so expensive,550,corgis expensive


## Poking around

In [188]:
demonym, r = s.head()

In [189]:
r

['why are the Indian so',
 ['what are the south indian states',
  'what are the south indian languages',
  'what are the southwest indian tribes',
  'what are the south indian dishes',
  'what are the south indian food',
  'what are the south indian tiffin items',
  'what are the south indian breakfast',
  'why indian song',
  'why south indian bank',
  'why south indian'],
 ['', '', '', '', '', '', '', '', '', ''],
 [],
 {'google:clientdata': {'bpc': False, 'tlw': False},
  'google:suggestrelevance': [601,
   600,
   557,
   556,
   555,
   554,
   553,
   552,
   551,
   550],
  'google:suggesttype': ['QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY',
   'QUERY'],
  'google:verbatimrelevance': 851}]

In [178]:
Counter([r[-1]['google:verbatimrelevance'] for r in s.values()])

Counter({851: 214, 1300: 1})

In [187]:
t = [r[-1].get('google:suggestrelevance', []) for r in s.values()]
pd.Series(Counter(map(len, t))).sort_values(ascending=False)

0     122
1      24
2      12
3       9
4       6
5       5
7       5
8       5
14      4
6       4
20      4
11      3
10      2
9       2
16      2
17      2
18      1
12      1
19      1
15      1
dtype: int64

# Getting the data

In [46]:
import requests
from bs4 import BeautifulSoup
import re

## List of demonyms

### From saved

In [250]:
# df = pd.read_excel('country_demonym.xlsx')
df = pd.read_pickle('country_demonym.p')
df

,country,demonym
0,Afghanistan,Afghan
1,Albania,Albanian
2,Algeria,Algerian
3,Andorra,Andorran
4,Angola,Angolan
...,...,...
212,Virgin Islands,Virgin Islander
213,Wales,Welsh
214,Yemen,Yemeni
215,Zambia,Zambian


### Scrape it

In [120]:
demonym_url = 'http://www.geography-site.co.uk/pages/countries/demonyms.html'

t = BeautifulSoup(requests.get(demonym_url).content)
tt = t.find_all('table')
tt = tt[-1]
tt = tt.find_all('tr')
len(tt)    

220

In [127]:
p = re.compile('\n|\r')

df = list()
for s in tt:
    s = s.text
    ss = list(filter(lambda x: len(x) > 1, p.split(s)))
    if ss:
        df.append(ss)
df = pd.DataFrame(df[1:], columns=list(map(str.lower, df[0])))
print(df.shape)
df.head()

(217, 2)


,country,demonym
0,Afghanistan,Afghan
1,Albania,Albanian
2,Algeria,Algerian
3,Andorra,Andorran
4,Angola,Angolan


In [249]:
# df.to_excel('country_demonym.xlsx', index=False)

In [143]:
p = re.compile('[\w\-\s]+')
demonyms = list(map(lambda x: p.match(x).group(0).strip(), df.demonym))

In [145]:
complicated_demonyms = list(filter(re.compile('\ or\ |\ \-\ |\ also\ ').findall, demonyms))
complicated_demonyms

['Azerbaijani - also Azeri',
 'Barbadian or Bajuns',
 'Cape Verdian or Cape Verdean',
 'Equatorial Guinean or Equatoguinean',
 'Grenadian or Grenadan',
 'Irishman or Irishwoman or Irish',
 'Lao or Laotian',
 'Monegasque or Monacan',
 'Burmese or Myanmarese',
 'Netherlander  or Dutch',
 'New Zealander or Kiwi',
 'Sammarinese or San Marinese',
 'Saudi or Saudi Arabian',
 'Serbian or Montenegrin',
 'Tajik or Tadzhik',
 'Trinidadian or Tobagonian',
 'Uzbek or Uzbekistani',
 'Yemeni or Yemenite']

In [146]:
{x: x for x in complicated_demonyms}

{'Azerbaijani - also Azeri': 'Azerbaijani - also Azeri',
 'Barbadian or Bajuns': 'Barbadian or Bajuns',
 'Cape Verdian or Cape Verdean': 'Cape Verdian or Cape Verdean',
 'Equatorial Guinean or Equatoguinean': 'Equatorial Guinean or Equatoguinean',
 'Grenadian or Grenadan': 'Grenadian or Grenadan',
 'Irishman or Irishwoman or Irish': 'Irishman or Irishwoman or Irish',
 'Lao or Laotian': 'Lao or Laotian',
 'Monegasque or Monacan': 'Monegasque or Monacan',
 'Burmese or Myanmarese': 'Burmese or Myanmarese',
 'Netherlander  or Dutch': 'Netherlander  or Dutch',
 'New Zealander or Kiwi': 'New Zealander or Kiwi',
 'Sammarinese or San Marinese': 'Sammarinese or San Marinese',
 'Saudi or Saudi Arabian': 'Saudi or Saudi Arabian',
 'Serbian or Montenegrin': 'Serbian or Montenegrin',
 'Tajik or Tadzhik': 'Tajik or Tadzhik',
 'Trinidadian or Tobagonian': 'Trinidadian or Tobagonian',
 'Uzbek or Uzbekistani': 'Uzbek or Uzbekistani',
 'Yemeni or Yemenite': 'Yemeni or Yemenite'}

In [147]:
replacements = {'Azerbaijani - also Azeri': 'Azerbaijani',
 'Barbadian or Bajuns': 'Barbadian',
 'Cape Verdian or Cape Verdean': 'Cape Verdian',
 'Equatorial Guinean or Equatoguinean': 'Equatorial Guinean',
 'Grenadian or Grenadan': 'Grenadian',
 'Irishman or Irishwoman or Irish ': 'Irish ',
 'Lao or Laotian': 'Laotian',
 'Monegasque or Monacan': 'Monegasque',
 'Burmese or Myanmarese': 'Burmese',
 'Netherlander  or Dutch ': 'Dutch ',
 'New Zealander or Kiwi': 'New Zealander',
 'Sammarinese or San Marinese': 'San Marinese',
 'Saudi or Saudi Arabian': 'Saudi',
 'Serbian or Montenegrin': 'Serbian',
 'Tajik or Tadzhik': 'Tajik',
 'Trinidadian or Tobagonian': 'Trinidadian',
 'Uzbek or Uzbekistani': 'Uzbek',
 'Yemeni or Yemenite': 'Yemeni'}
replace = lambda x: replacements.get(x, x)

In [152]:
df['demonym'] = list(map(replace, demonyms))
df

,country,demonym
0,Afghanistan,Afghan
1,Albania,Albanian
2,Algeria,Algerian
3,Andorra,Andorran
4,Angola,Angolan
...,...,...
212,Virgin Islands,Virgin Islander
213,Wales,Welsh
214,Yemen,Yemeni
215,Zambia,Zambian


In [506]:
# df.to_excel('country_demonym.xlsx', index=False)
# df.to_pickle('country_demonym.p')

## Auto-suggest

In [251]:
import requests, json, re

p_filename_chars = re.compile('[^\w\-]')

def filename_of_demonym(demonym):
    t = demonym.strip().lower()
    t = p_filename_chars.sub('_', t)
    return f"{t}.json"

def google_auto_suggestions_response(q, user_agent='Chrome/79.0.3945.88', 
                            url_prefix='http://suggestqueries.google.com/complete/search?client=chrome&q='):
    url = f"{url_prefix}{q}"
    headers = {'User-agent': user_agent}
    return requests.get(url, headers=headers)

def google_auto_suggestions(q, user_agent='Chrome/79.0.3945.88', 
                            url_prefix='http://suggestqueries.google.com/complete/search?client=chrome&q='):
    try:
        response = google_auto_suggestions_response(q, user_agent, url_prefix)
        result = json.loads(response.content.decode('utf-8'))
        return result[1]
    except Exception as e:
        print(f"{e}")
        print("Returning raw response")
        return response

query_format_1 = "why are the {demonym} so "  # space after the "so" is important!
query_format_2 = "why are the {demonym} "  # more hits than so, but many not targeted to PEOPLE
query_format_3 = "why are {demonym} so " # more hits than so, but many not targeted to PEOPLE 


query_format = query_format_1

def demonym_adjectives_response(demonym):
    return google_auto_suggestions_response(q=query_format.format(demonym=demonym))

def demonym_adjectives_json(demonym):
    response = demonym_adjectives_response(demonym)
    return json.loads(response.content.decode('utf-8'))

def demonym_adjectives(demonym):
    return google_auto_suggestions(q=query_format.format(demonym=demonym))
    

In [252]:
from py2store import *
import os

In [259]:
root = os.path.expanduser('~/ddir/demonym_adjectives/so/')
s = QuickJsonStore(root)
len(s)

215

In [254]:
import time

for i, demonym in enumerate(df['demonym']):
    print(f"{i}: {demonym}")
    s[demonym] = demonym_adjectives_json(demonym)
    time.sleep(0.5 + 1.5 * random.random())

0: Afghan
1: Albanian
2: Algerian
3: Andorran
4: Angolan
5: Anguillan
6: Antiguans
7: Argentine
8: Armenian
9: Australian
10: Austrian
11: Azerbaijani
12: Bahamian
13: Bahraini
14: Bangladeshi
15: Barbadian
16: Belarusian
17: Belgian
18: Belizean
19: Beninese
20: Bermudian
21: Bhutanese
22: Bolivian
23: Bosnian
24: Motswana
25: Brazilian
26: British Virgin Islander
27: Bruneian
28: Bulgarian
29: Burkinabe
30: Burundian
31: Cambodian
32: Cameroonian
33: Canadian
34: Cape Verdian
35: Caymanian
36: Central African
37: Chadian
38: Chilean
39: Chinese
40: Christmas Islander
41: Cocos Islander
42: Colombian
43: Comoran
44: Congolese
45: Congolese
46: Costa Rican
47: Ivorian
48: Croat
49: Cuban
50: Cypriot
51: Czech
52: Dane
53: Djibouti
54: Dominican
55: Dominican
56: East Timorese
57: Ecuadorean
58: Egyptian
59: Salvadoran
60: English
61: Equatorial Guinean
62: Eritrean
63: Estonian
64: Ethiopian
65: Falkland Islander
66: Fijian
67: Finn
68: French
69: French Guianese
70: Gabonese
71: Gambi

In [515]:
demonym = 'Serbian'
t = demonym_adjectives_json(demonym)
t

['why are the Serbian so ',
 ['why serbian so tall'],
 [''],
 [],
 {'google:clientdata': {'bpc': False, 'tlw': False},
  'google:suggestrelevance': [600],
  'google:suggesttype': ['QUERY'],
  'google:verbatimrelevance': 851}]

In [517]:
s['Serbian']

['why are the Serbian so ',
 ['why serbian so tall'],
 [''],
 [],
 {'google:clientdata': {'bpc': False, 'tlw': False},
  'google:suggestrelevance': [600],
  'google:suggesttype': ['QUERY'],
  'google:verbatimrelevance': 851}]